In [1]:
import os
import glob
import methpype
import pandas as pd
import numpy as np

In [2]:
data_dir = "/data/Genetic/tab_data"

In [48]:
cellc = pd.read_csv("/data/Genetic/tab_data/EstimateCellCounts_PPMI_EPICn524final030618.csv")
pat_stat = pd.read_csv("/data/Genetic/tab_data/Patient_Status.csv")
pat_info = pd.read_csv("/data/Genetic/tab_data/PPMI_Meth_n524_for_LONI030718.tsv", delimiter='\t')
raw_dat = pd.read_csv("/data/Genetic/tab_data/beta_post_Funnorm_PPMI_EPICn524final030618.csv")

In [4]:
pat_numbers = list(pat_info["PATNO"].astype(int))
is_PD = lambda id: pat_stat[pat_stat.PATNO == id]["RECRUITMENT_CAT"].values[0] == 'PD'
get_sentrix = lambda id: str(pat_info[pat_info.PATNO == id]["Sentrix ID"].values[0]) + "_" + pat_info[pat_info.PATNO == id]["Sentrix Position"].values[0]
getXVal_cellc = lambda id: cellc[cellc.Sentrix_position == get_sentrix(id)].values[0][1:].astype(float)
getXVal_raw = lambda id: raw_dat[get_sentrix(id)].values

In [6]:
X = np.zeros((len(pat_numbers), 864067))
y = np.zeros((len(pat_numbers)), dtype=int)
for i, number in enumerate(pat_numbers):
    if(is_PD(number)):
        y[i] = 1
    X[i] = getXVal_raw(number)
raw_dat = None
import gc
gc.collect()

7

In [9]:
from sklearn.decomposition import PCA as pca
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split as splt
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import matthews_corrcoef as mcc

forest = rfc(n_estimators = 1000, n_jobs = -1)
decomp = pca(n_components=32, svd_solver='full')
X_tr, X_ts, y_tr, y_ts = splt(X, y, test_size = 0.2, random_state = 42, stratify = y)
decomp.fit(X_tr)
X_tr_decomp = decomp.transform(X_tr)
X_ts_decomp = decomp.transform(X_ts)
forest.fit(X_tr_decomp, y_tr)
y_ts_pred = forest.predict(X_ts_decomp)

In [14]:
forest.feature_importances_

array([0.03458826, 0.03694026, 0.03380485, 0.03884156, 0.04747703,
       0.03714337, 0.03303851, 0.0367736 , 0.02699897, 0.03311466,
       0.03495015, 0.0325671 , 0.0361581 , 0.02797452, 0.02945479,
       0.02619802, 0.02826112, 0.02840834, 0.03357868, 0.02822882,
       0.02525512, 0.02838569, 0.02311755, 0.03250804, 0.04333419,
       0.02663297, 0.02695711, 0.02689515, 0.02411678, 0.02526081,
       0.02444954, 0.02858635])

In [31]:
temp_arr = decomp.explained_variance_ * forest.feature_importances_

In [30]:
decomp.components_.shape

(32, 864067)

In [55]:
feature_importances = np.matmul(temp_arr.reshape((1,32)), decomp.components_)[0]

In [53]:
raw_dat.index.values.shape

(864067,)

In [56]:
feature_importances.shape

(864067,)

In [60]:
feature_dic = dict(zip(raw_dat.index.values, abs(feature_importances)))

In [61]:
feature_dic

{'cg14817997': 0.01660302567912256,
 'cg26928153': 0.0064495307534584495,
 'cg16269199': 0.021527915897247736,
 'cg13869341': 0.002462788821454595,
 'cg14008030': 0.012043911233495335,
 'cg12045430': 0.009734536066795654,
 'cg20826792': 0.0006109091555481496,
 'cg20253340': 0.015197295163502085,
 'cg02404219': 0.05130448089682942,
 'cg21870274': 0.016551504307169278,
 'cg04098293': 0.014559808984885245,
 'cg16382250': 0.01147074876628397,
 'cg24335620': 0.001401529667190521,
 'cg22463138': 0.0053197886295917466,
 'cg16162899': 0.032612368299116345,
 'cg01803908': 0.0011675719357638606,
 'cg17149495': 0.0322222734145263,
 'cg22802167': 0.002173629817258939,
 'cg24669183': 0.003633701628052519,
 'cg17308840': 0.005796814524524868,
 'cg17866181': 0.018296641161648322,
 'cg25018832': 0.00013847851215188084,
 'cg26679879': 0.01877673985155291,
 'cg22519184': 0.013000728317413376,
 'cg11894324': 0.0006853680181903646,
 'cg24159721': 0.001915129270365162,
 'cg02288058': 0.014376880624021492,


In [74]:
import operator
sorted_features_value = sorted(feature_dic.items(), key = lambda item: -1 * item[1])

In [75]:
sorted_features = [feature[0] for feature in sorted_features_value]

In [77]:
sorted_features_value

[('cg15925199', 0.09889042790726574),
 ('cg25205946', 0.09814122170826359),
 ('cg10252249', 0.0953947574394675),
 ('cg08417382', 0.0945028579044499),
 ('cg27277239', 0.09363280317152597),
 ('cg22028367', 0.09320392630211613),
 ('cg17544920', 0.09183219873750198),
 ('cg23680829', 0.09170028917876996),
 ('cg17797028', 0.09135508865824363),
 ('cg04820593', 0.09125157850519822),
 ('cg07851521', 0.09083209032984638),
 ('cg16767700', 0.09060017271719724),
 ('cg16764023', 0.08911644832538253),
 ('cg14530679', 0.08902395207449423),
 ('cg26439324', 0.08884066078918605),
 ('cg14804181', 0.08860577731484512),
 ('cg15774752', 0.0885091771036988),
 ('cg24446128', 0.08825801329392924),
 ('cg12878951', 0.08823702766509432),
 ('cg23762217', 0.08821887334050067),
 ('cg15468246', 0.08820757059296366),
 ('cg01807421', 0.08775780706211311),
 ('cg00282877', 0.08759720337859547),
 ('cg03685263', 0.08726521222119814),
 ('cg27488875', 0.08717846241192473),
 ('cg06367887', 0.08676175803326482),
 ('cg09735782',